# 환경변수 로딩

In [11]:
from dotenv import load_dotenv
import os
load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## RunnableLambda 객체 사용방법

In [12]:
from langchain_core.runnables import RunnableLambda

# 함수 정의
def func(x):
    return  x*2

# 함수를 전달인자로 넣기
runnable_1 = RunnableLambda(func)

# RunnableLambda를 통한 함수 실행
print(runnable_1.invoke(10))

20


In [13]:
# RunnableLambda 객체 사용
runnable_1 = RunnableLambda(lambda x: x*2)
print(runnable_1.invoke(10))

20


In [14]:
# runnable batch
runnable_3 = RunnableLambda(func)

print(runnable_3.batch([10, 20, 30]))

[20, 40, 60]


## 순차적으로 실행하게 하기

In [15]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

chain = r1 | r2

chain.invoke(10)



35

# 

In [16]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# chain = r1 | r2  # 아래와 같은 표현
chain = RunnableSequence(r1, r2)

chain.invoke(10)

35

In [17]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# 병렬처리 , r1, r2 키는 임으로 설정해 주면 됨.
# chain = RunnableParallel(r1=r1, r2=r2)
chain = RunnableParallel(first=r1, second=r2)

chain.invoke(10)

{'first': 30, 'second': 15}

In [18]:
# 그래프 그리기 알고리즘 
# pip install -qU grandalf

In [19]:
[{"foo":2}]*3

[{'foo': 2}, {'foo': 2}, {'foo': 2}]

In [20]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable_1 = RunnableLambda(lambda x: {"foo": x})
runnable_2 = RunnableLambda(lambda x: [x] * 3)
runnable_3 = RunnableLambda(lambda x: str(x))

chain = runnable_1 | RunnableParallel(r2=runnable_2, r3=runnable_3)

print(chain.invoke(2))
chain.get_graph().print_ascii()

{'r2': [{'foo': 2}, {'foo': 2}, {'foo': 2}], 'r3': "{'foo': 2}"}
        +-------------+        
        | LambdaInput |        
        +-------------+        
               *               
               *               
               *               
          +--------+           
          | Lambda |           
          +--------+           
               *               
               *               
               *               
   +----------------------+    
   | Parallel<r2,r3>Input |    
   +----------------------+    
          *         *          
        **           **        
       *               *       
+--------+          +--------+ 
| Lambda |          | Lambda | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
  +-----------------------+    
  | Parallel<r2,r3>Output |    
  +-----------------------+    


## LCEL 문법 적용

- 방법1

In [21]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

In [22]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = prompt | llm
input = {
            "context": "Vector Search에 의한 컨텐스트 내용", 
            "query": "안녕"
        }
print(chain.invoke(input))

content='안녕하세요! 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 42, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CuJdAnuJUTKGezTch8n3743InGhCd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b8977-6a58-76f1-ab0d-902e2cb0b857-0' usage_metadata={'input_tokens': 42, 'output_tokens': 10, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


- 방법2 RunnableLambda 사용

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

# 기본값을 제공하는 함수
def add_default_values(input_dict):
    return {
        "context": input_dict.get("context", "Vector Search에 의한 컨텐스트 내용"),
        "query": input_dict.get("query", "안녕")
    }


# 체인 구성
chain = RunnableLambda(add_default_values) | prompt | llm


In [24]:
# 빈 딕셔너리로도 실행 가능
result = chain.invoke({})
print(result)

content='안녕하세요! 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 42, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CuJdA2tB6cezrXg6hdtS9v9s2OY93', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b8977-6e0d-7343-b9e9-c9bfb0d4b23f-0' usage_metadata={'input_tokens': 42, 'output_tokens': 10, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [25]:
# 또는 일부 값만 override
result2 = chain.invoke({"query": "RunnableLambda에 대해 1문장으로 설명해줘"})


In [26]:
print(result2.content)

RunnableLambda는 Java에서 람다 표현식을 사용하여 Runnable 인터페이스를 구현하는 간결한 방법을 제공하는 기능입니다.


- Runnable 객체를 만들면 pipeline에 태울 수 있다는 것을 아는게 중요함.

# 실행을 그래프로 시각화 하기

In [27]:
chain.get_graph().print_ascii()

+--------------------------+ 
| add_default_values_input | 
+--------------------------+ 
              *              
              *              
              *              
   +--------------------+    
   | add_default_values |    
   +--------------------+    
              *              
              *              
              *              
   +--------------------+    
   | ChatPromptTemplate |    
   +--------------------+    
              *              
              *              
              *              
       +------------+        
       | ChatOpenAI |        
       +------------+        
              *              
              *              
              *              
    +------------------+     
    | ChatOpenAIOutput |     
    +------------------+     
